<h1>Modelagem de dados sequenciais usando redes neurais recorrentes</h1>

<p align=center><img src=https://datascience.eu/wp-content/uploads/2020/05/image-513-1024x347.png></p>

Tivemos oportunidade de focar em redes neurais convolucionais (*CNNs*), de forma a cobrir os blocos de construção das arquiteturas *CNN* e como implementar *CNNs* profundas no *TensorFlow*. Por fim, você aprendeu a usar *CNNs* para classificação de imagens.

Aqui, exploraremos as redes neurais recorrentes (*RNNs*) e veremos sua aplicação na modelagem de dados sequenciais.

### Apresentando dados sequenciais
Vamos começar nossa discussão sobre *RNNs* observando a natureza dos dados sequenciais, que são mais comumente conhecidos como dados de sequência ou **sequências**. Vamos dar uma olhada nas propriedades únicas das sequências que as tornam diferentes de outros tipos de dados. Veremos então como podemos representar dados sequenciais e explorar as várias categorias de modelos para dados sequenciais, que são baseados na entrada e saída de um modelo. Isso nos ajudará a explorar a relação entre *RNNs* e sequências.

### Modelagem de dados sequenciais - a ordem importa

O que torna as sequências únicas, em comparação com outros tipos de dados, é que os elementos em uma sequência aparecem em uma determinada ordem e não são independentes uns dos outros. Algoritmos típicos de aprendizado de máquina para aprendizado supervisionado pressupõem que a entrada de dados é **independente e e distribuída de forma idêntica (IID)**, o que significa que os exemplos de treinamento são `mutuamente independentes` e têm a mesma distribuição subjacente.

Nesse sentido, com base na suposição de independência mútua, a ordem em que os exemplos de treinamento são dados ao modelo é **irrelevante**. Por exemplo, se tivermos uma amostra composta por n exemplos de treinamento, $\small x^{(1)}, x^{(2)},\cdots, x^{(n)} $, a ordem em que usamos os dados para treinar nosso algoritmo de aprendizado de máquina não importa. Um exemplo desse cenário seria o conjunto de dados Iris, muito conhecido. No conjunto de dados Iris, cada flor foi medida independentemente e as medidas de uma flor não influenciam as medidas de outra flor.

No entanto, essa suposição não é válida quando lidamos com sequências – por definição, **a ordem é importante**. Prever o valor de mercado de uma determinada ação seria um exemplo desse cenário. Por exemplo, suponha que temos uma amostra de n exemplos de treinamento, onde cada exemplo de treinamento representa o valor de mercado de uma determinada ação em um determinado dia. Se nossa tarefa é prever o valor do mercado de ações para os próximos três dias, faria sentido considerar os preços das ações anteriores em uma ordem de data para derivar tendências, em vez de utilizar esses exemplos de treinamento em uma ordem aleatória.

> #### Dados sequenciais versus dados de séries temporais
> Os dados de série temporal são um tipo especial de dados sequenciais, em que cada exemplo está associado a uma dimensão de tempo. Em dados de séries temporais, as amostras são coletadas em *timestamps* sucessivos e, portanto, a dimensão de tempo determina a ordem entre os pontos de dados. Por exemplo, preços de ações e registros de voz ou fala são dados de séries temporais.
>
> Por outro lado, nem todos os dados sequenciais têm a dimensão temporal, por exemplo, dados de texto ou sequências de DNA, onde os exemplos são ordenados, mas não se qualificam como dados de séries temporais. Como você verá, abordaremos alguns exemplos de Processamento de Linguagem Natural (NLP) e modelagem de texto que não são dados de série temporal, mas observe que as *RNNs* também podem ser usados para dados de série temporal.

### Representando sequências

Estabelecemos que a ordem entre os pontos de dados é importante em dados sequenciais, então precisamos encontrar uma maneira de aproveitar essas informações de pedido em um modelo de aprendizado de máquina. Ao longo das explicações, representaremos as sequências como $\small \left \langle x^{(1)},x^{(2)},\cdots, x^{(T)}  \right \rangle$ . Os índices sobrescritos indicam a ordem das instâncias e o comprimento da sequência é $\small T$. Para um exemplo sensato de sequências, considere dados de séries temporais, onde cada ponto de exemplo, $\small x^{(T)}$, pertence a um determinado tempo, $\small t$. A figura a seguir mostra um exemplo de dados de série temporal em que os recursos de entrada ($\small x$) e os rótulos de destino ($\small y$) seguem naturalmente a ordem de acordo com seu eixo de tempo; portanto, ambos os `x` e `y` são sequências:

![](imagens\sequencias.PNG)

Como já mencionamos, os modelos de rede neural padrão (*RN*) que abordamos até agora, como o *multilayer perceptron* (MLP) e as *CNNs* para dados de imagem, assumem que os exemplos de treinamento são independentes uns dos outros e, portanto, não incorporam **informação de ordenamento**. Podemos dizer que tais modelos não possuem **memória** de exemplos de treinamento vistos anteriormente. Por exemplo, as amostras são passadas pelas etapas de *feedforward* e *backpropagation* e os pesos são atualizados independentemente da ordem em que os exemplos de treinamento são processados. As *RNNs*, por outro lado, são projetadas para modelar sequências e são capazes de lembrar informações passadas e processar novos eventos de acordo, o que é uma clara vantagem ao trabalhar com dados de sequência.

### As diferentes categorias de modelagem de sequência

A modelagem de sequência tem muitas aplicações fascinantes, como tradução de idiomas (por exemplo, tradução de texto de inglês para alemão), legendas de imagens e geração de texto. No entanto, para escolher uma arquitetura e abordagem apropriadas, temos que entender e ser capazes de distinguir entre essas diferentes tarefas de modelagem de sequência. A figura a seguir, baseada nas explicações do excelente artigo The Unreasonable Effectiveness of Recurrent Neural Networks, de Andrej Karpathy (http://karpathy.github.io/2015/05/21/rnn-effectiveness/), resume a sequência mais comum tarefas de modelagem, que dependem das categorias de relacionamento de dados de entrada e saída:

![](imagens\modelagem_de_sequencia.PNG)

Vamos discutir as diferentes categorias de relacionamento entre dados de entrada e saída, que foram descritas na figura anterior, com mais detalhes. Se nem os dados de entrada nem de saída representam sequências, então estamos lidando com dados padrão e podemos simplesmente usar um perceptron multicamada (ou outro modelo de classificação abordado anteriormente) para modelar esses dados. No entanto, se a entrada ou a saída for uma sequência, a tarefa de modelagem provavelmente se enquadra em uma destas categorias:
* **Muitos para um**: Os dados de entrada são uma sequência, mas a saída é um vetor de tamanho fixo ou escalar, não uma sequência. Por exemplo, na análise de sentimentos, a entrada é baseada em texto (por exemplo, uma resenha de filme) e a saída é um rótulo de classe (por exemplo, um rótulo que indica se um revisor gostou do filme).

* **Um para muitos**: Os dados de entrada estão no formato padrão e não em sequência, mas a saída é uma sequência. Um exemplo dessa categoria é a legendagem de imagens — a entrada é uma imagem e a saída é uma frase em inglês que resume o conteúdo dessa imagem.

* **Muitos para muitos**: As matrizes de entrada e saída são sequências. Esta categoria pode ser dividida com base na sincronização da entrada e saída. Um exemplo de uma tarefa de modelagem sincronizada de muitos para muitos é a classificação de vídeo, onde cada quadro em um vídeo é rotulado. Um exemplo de uma tarefa de modelagem muitos-para-muitos *atrasada* seria traduzir uma linguagem para outra. Por exemplo, uma frase inteira em inglês deve ser lida e processada por uma máquina antes que sua tradução para o alemão seja produzida. 

Agora, depois de resumir as três grandes categorias de modelagem de sequência, podemos avançar para discutir a estrutura de uma RNN.

### RNNs para modelagem de sequências 
Nesta seção, antes de começarmos a implementar *RNNs* no *TensorFlow*, discutiremos os principais conceitos de *RNNs*. Começaremos examinando a estrutura típica de uma *RNN*, que inclui um componente recursivo para modelar dados de sequência. Em seguida, examinaremos como as ativações dos neurônios são computadas em uma *RNN* típica. Isso criará um contexto para discutirmos os desafios comuns no treinamento de *RNNs* e, em seguida, discutiremos soluções para esses desafios, como *LSTM* e unidades recorrentes fechadas (*GRUs*).

### Entendendo o mecanismo de loop *RNN*
Vamos começar com a arquitetura de uma *RNN*. A figura a seguir mostra uma *RN* *feedforward* padrão e um *RNN* lado a lado para comparação:

![](imagens\mecanismo_loop_rnn.PNG)

Ambas as redes têm apenas uma camada oculta. Nesta representação, as unidades não são exibidas, mas assumimos que a camada de entrada ($\small x$), a camada oculta ($\small h$) e a camada de saída ($\small o$) são vetores que contêm muitas unidades.

> ##### Determinando o tipo de saída de um RNN
> Essa arquitetura RNN genérica pode corresponder às duas categorias de modelagem de sequência em que a entrada é uma sequência. Normalmente, uma camada recorrente pode retornar uma sequência como saída, $\small \left \langle o^{(1)},o^{(2)},\cdots, o^{(T)}  \right \rangle$, ou simplesmente retornar a última saída (em $\small t = T$, ou seja, $\small o^{(T)}$). Assim, pode ser muitos para muitos ou muitos para um se, por exemplo, usarmos apenas o último elemento, $\small o^{(T)}$, como a saída final.
>
> Como você verá mais tarde, na *API TensorFlow Keras*, o comportamento de uma camada recorrente em relação ao retorno de uma sequência como saída ou simplesmente usar a última saída pode ser especificado definindo o argumento `return_sequences` como `True` ou `False`, respectivamente.

Em uma rede *feedforward* padrão, as informações fluem da entrada para a camada oculta e, em seguida, da camada oculta para a camada de saída. Por outro lado, em uma *RNN*, a camada oculta recebe sua entrada tanto da camada de entrada da etapa de tempo atual quanto da camada oculta da etapa de tempo anterior.

O fluxo de informações em etapas de tempo adjacentes na camada oculta permite que a rede tenha uma memória de eventos passados. Esse fluxo de informações geralmente é exibido como um *loop*, também conhecido como **recurrent edge** (borda recorrente) em notação de gráfico, que é como essa arquitetura *RNN* geral recebeu seu nome.

Semelhante aos *perceptrons* multicamadas, as *RNN*s podem consistir em várias camadas ocultas. Observe que é uma convenção comum se referir às *RNNs* com uma camada oculta como uma *RNN* de camada única, que não deve ser confundido com as RNs de camada única sem uma camada oculta, como *Adaline* ou *regressão logística*. A figura a seguir ilustra uma *RNN* com uma camada oculta (superior) e uma *RNN* com duas camadas ocultas (inferior):

![](imagens\rnn_oculta.PNG)

Para examinar a arquitetura das *RNNs* e o fluxo de informações, pode-se desdobrar uma representação compacta com uma aresta recorrente, que você pode ver na figura anterior.

Como sabemos, cada unidade oculta em uma *RN* padrão recebe apenas uma entrada – a pré-ativação de rede associada à camada de entrada. Em contraste, cada unidade oculta em uma *RNN* recebe dois conjuntos distintos de entrada – a pré-ativação da camada de entrada e a ativação da mesma camada oculta da etapa de tempo anterior, $\small t – 1$.

Na primeira etapa de tempo, $\small t = 0$, as unidades ocultas são inicializadas com zeros ou pequenos valores aleatórios. Então, em um passo de tempo em que $\small t > 0$, as unidades ocultas recebem sua entrada do ponto de dados no momento atual, $\small x^{(T)}$, e os valores anteriores das unidades ocultas em $\small t – 1$, indicados como $\small h^{(t-1)}$.